In [2]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt

In [36]:
def center_crop(image, crop_size):
    """
    对输入图像进行中心裁剪。

    参数：
        image: 输入图像
        crop_size: 裁剪尺寸 (width, height)

    返回：
        中心裁剪后的图像
    """
    if crop_size[0] >= image.shape[1] or crop_size[1] >= image.shape[0]:
        raise ValueError("裁剪尺寸大于图像尺寸")

    center_x = image.shape[1] // 2
    center_y = image.shape[0] // 2

    half_width = crop_size[0] // 2
    half_height = crop_size[1] // 2

    cropped = image[center_y - half_height:center_y + half_height,
                    center_x - half_width:center_x + half_width]

    return cropped

In [74]:
im_bgr = cv2.imread("./pic_all/1-1_rId4_0.png")  # 原始的彩色图像文件，BGR模式
# 定义目标尺寸
target_size = (660, 660)

# 调整图像大小
im_bgr = cv2.resize(im_bgr, target_size)
im_gray = cv2.cvtColor(im_bgr, cv2.COLOR_BGR2GRAY)  # 转灰度图像
im_gray = cv2.GaussianBlur(im_gray, (3, 3), 0)  # 灰度图像滤波降噪
im_edge = cv2.Canny(im_gray, 30, 50)  # 边缘检测获得边缘图像
print(im_gray.shape)

(660, 660)


In [75]:
def split_image(image, rows, cols):
    height, width = image.shape[:2]
    cell_height = height // rows
    cell_width = width // cols

    cells = []
    for row in range(rows):
        for col in range(cols):
            y1 = row * cell_height
            y2 = (row + 1) * cell_height
            x1 = col * cell_width
            x2 = (col + 1) * cell_width

            cell = image[y1:y2, x1:x2]
            cells.append(cell)

    return cells


In [76]:
def analyze_image(image):
    height, width, _ = image.shape
    total_pixels = height * width

    # 计算黑色、白色和特定颜色的像素数量
    black_pixels = np.sum(np.logical_and.reduce(image < 10, axis=-1))
    white_pixels = np.sum(np.logical_and.reduce(image > 190, axis=-1))
    specific_color_pixels = np.sum(
        np.logical_and(
            np.logical_and(image[:, :, 2] >= 200, image[:, :, 2] <= 220),
            np.logical_and(image[:, :, 1] >= 170, image[:, :, 1] <= 190),
            np.logical_and(image[:, :, 0] >= 130, image[:, :, 0] <= 150),
        )
    )

    # 计算像素占比
    black_ratio = black_pixels / total_pixels
    white_ratio = white_pixels / total_pixels
    specific_color_ratio = specific_color_pixels / total_pixels

    # 根据占比判断图片类别
    if black_ratio > 0.3:
        return 1
    elif white_ratio > 0.2:
        return 2
    elif specific_color_ratio > 0.5:
        return 0
    else:
        return 0

In [77]:
crop_size = (608, 608)  # 替换为所需的裁剪尺寸
cropped_image = center_crop(im_bgr, crop_size)
# 将裁剪后的图像分割为19x19个小格子
split_rows = 19
split_cols = 19
small_cells = split_image(cropped_image, split_rows, split_cols)
# 创建一个空矩阵来保存类别信息
class_matrix = np.zeros((split_rows, split_cols), dtype=int)
print(len(small_cells))
# 对每个小格子应用判断函数，得到类别信息
row = 0
cols = 0
for i in small_cells:
    class_matrix[cols][row] = analyze_image(i)
    row += 1
    if row ==19 :
        row = 0
        cols += 1
    print(analyze_image(i))

print("Class Matrix:")
print(class_matrix)

361
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
2
0
0
2
0
0
0
0
0
0
1
0
0
0
0
0
0
1
0
0
0
2
0
0
1
0
2
0
2
1
0
1
0
0
0
0
1
2
0
1
1
2
0
0
1
0
0
2
1
0
0
0
0
0
0
0
2
0
0
0
2
1
2
1
0
2
0
2
0
0
0
0
0
0
1
2
0
0
1
0
1
1
2
2
0
2
0
0
0
0
0
0
0
1
2
0
1
1
0
1
2
0
0
0
0
0
0
0
0
0
0
1
2
0
0
0
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
2
0
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
2
1
0
0
1
2
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
1
2
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
1
2
0
0
0
0
0
0
0
0
0
2
0
1
0
0
0
0
0
1
2
0
0
0
0
0
2
0
0
0
0
2
2
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
Class Matrix:
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 2 0 0 2 0 0 0 0 0 0 1 0 0 0 0]
 [0 0 1 0 0 0 2 0 0 1 0 2 0 2 1 0 1 0 0]
 [0 0 1 2 0 1 1 2 0 0 1 0 0 2 1 0 0 0 0]
 [0 0 0 2 0 0 0 2 1 2 1 0 2 0 2 0 0 0 0]
 [0 0 1 2 0 0 